In [ ]:
import os
import random 
import pandas as pd
import warnings
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from loguru import logger

# local imports
sys.path.append(r"./utils")
from utils import utils

random.seed(1234)
warnings.filterwarnings('ignore')

In [ ]:
COLAB = False
SAMPLE = True
DRY_RUN = False
DATA_DIR = "./data/full/"
GROUP_NAME = "Self_Reported_Segments"
MODEL_LIST = ["NN", "LR"]
RESULTS_DIR = ".results/full/Self_Reported_Segments/"
RESULTS_STATS_FILENAME = GROUP_NAME + '.csv'
RESULTS_MODEL_FILENAME_PREFIX = GROUP_NAME
RESULTS = []

MODEL_RESULTS = './results/Model results/'

logger.debug(f"Started the script for {GROUP_NAME}.")

if SAMPLE:
  DATA_DIR = "./data/sample/"
  RESULTS_DIR = "./results/sample/Self_Reported_Segments/"

DATA_DROP_COLS = \
           ['Unnamed: 0', # index columns
           'Unnamed: 0.1', # TODO: might need to change this for bigger dataset. ,
              # 'pol',  # label column
              'gender', # self reported and filtered already
              # 'age', # self-reported  
              'country', # self reported and filtered already
              'userid', # index equivalent column 
              'pol_dat_us', # redundant columns with label
              'pol_dat_ca', # redundant columns with label
              'pol_dat_uk', # redundant columns with label
              'pol_fb_us', # redundant columns with label
              'database', # filtered already 
              ]

In [ ]:
if COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
    
  #TODO: untested for COLAB == True, more might be needed here
  DATA_DIR = "/content/drive/Shareddrives/Facial Recognition/data/"

In [4]:
# List of data file paths

folders = os.listdir(DATA_DIR)

for folder in tqdm(folders):
  
  if folder != "NO FILES":

    data = utils.get_segment_dataframe(data_dir = DATA_DIR, segment_to_run = folder)

    # Clean the data
    try:
      data = data.drop(DATA_DROP_COLS, axis=1)
    except:
      pass

    data = utils.get_clean_data(data)
    data_y = data['pol'].replace({"liberal": 1, "conservative": 0})
    data = data.drop('pol', axis = 1)
    all_features = data.columns

    X_train, X_test, y_train, y_test = train_test_split(data, data_y, test_size = 0.2, random_state=1234) 
    # TODO: check reproducibility of splits so that additional metrics computed post-hoc
    # are consistent. See https://stackoverflow.com/questions/53182821/scikit-learn-train-test-split-not-reproducible

    del data, data_y

    # Define features for the various settings
    image_cols = list(map(str, range(1, 2049)))
    self_reported_cols = ['age']
    image_and_self_reported_cols = image_cols + self_reported_cols
    image_and_extracted_cols = [x for x in all_features if x not in self_reported_cols]
    image_and_self_reported_and_extracted_cols = all_features

    # which group it is being processed on 
    SEGMENT_NAME = folder
    logger.debug(f"Started the script for {SEGMENT_NAME}.")

    data_dict = {
                "Image Features" : image_cols,
                "Image and Self Reported Features" : image_and_self_reported_cols,
                "Image and Extracted Features": image_and_extracted_cols,
                "Image, Self-reported and Extracted Features": image_and_self_reported_and_extracted_cols
                }

    # Fit models
    for data_name, data_set_features in tqdm(data_dict.items()):
      for model_name in MODEL_LIST:
        try:
          logger.debug(f"{SEGMENT_NAME}, {model_name}, {data_name}: model training started.")
          auc, acc, model = utils.fit_and_get_metrics(model_name,
                                                      X_train[data_set_features],
                                                      y_train,
                                                      X_test[data_set_features],
                                                      y_test,
                                                      dry_run = DRY_RUN)
          utils.save_model(model, model_name, RESULTS_DIR \
                          + SEGMENT_NAME \
                          + "_" + model_name \
                          + "_" + data_name.replace(" ","_").replace(",","").replace("-","_") \
                          + '.mdl')
          RESULTS.append([SEGMENT_NAME, model_name, data_name, auc, acc])
          logger.debug(f"{SEGMENT_NAME}, {model_name}, {data_name}: model training ended. AUC: {auc}, accuracy: {acc}")
        except:
          logger.debug(f"{SEGMENT_NAME}, {model_name}, {data_name}: Error occured!")
    
utils.save_results(results_array = RESULTS, location = MODEL_RESULTS + RESULTS_STATS_FILENAME)
logger.debug(f"Script for {GROUP_NAME} finished.")
    

Epoch 16/25
2560/2560 [==============================] - 0s 135us/step - loss: 0.5240 - accuracy: 0.7473 - val_loss: 0.5530 - val_accuracy: 0.7312
Epoch 17/25
2560/2560 [==============================] - 0s 128us/step - loss: 0.5125 - accuracy: 0.7348 - val_loss: 0.5534 - val_accuracy: 0.7312
Epoch 18/25
2560/2560 [==============================] - 0s 137us/step - loss: 0.5229 - accuracy: 0.7473 - val_loss: 0.5651 - val_accuracy: 0.7344
Epoch 19/25
2560/2560 [==============================] - 0s 129us/step - loss: 0.5208 - accuracy: 0.7395 - val_loss: 0.5602 - val_accuracy: 0.7359
Epoch 20/25
2560/2560 [==============================] - 0s 132us/step - loss: 0.5130 - accuracy: 0.7547 - val_loss: 0.5502 - val_accuracy: 0.7328
Epoch 21/25
2560/2560 [==============================] - 0s 130us/step - loss: 0.5001 - accuracy: 0.7422 - val_loss: 0.5449 - val_accuracy: 0.7250
Epoch 22/25
2560/2560 [==============================] - 0s 129us/step - loss: 0.5001 - accuracy: 0.7523 - val_loss: 0

2021-08-15 15:59:48.849 | DEBUG    | __main__:<module>:63 - US_1_FB, NN, Image and Extracted Features: model training ended. AUC: 62.28, accuracy: 70.38
2021-08-15 15:59:48.849 | DEBUG    | __main__:<module>:50 - US_1_FB, LR, Image and Extracted Features: model training started.
2021-08-15 15:59:54.635 | DEBUG    | __main__:<module>:63 - US_1_FB, LR, Image and Extracted Features: model training ended. AUC: 70.14, accuracy: 74.12
2021-08-15 15:59:54.637 | DEBUG    | __main__:<module>:50 - US_1_FB, NN, Image, Self-reported and Extracted Features: model training started.


Train on 2560 samples, validate on 640 samples
Epoch 1/25
2560/2560 [==============================] - 1s 325us/step - loss: 0.6937 - accuracy: 0.6313 - val_loss: 1.2899 - val_accuracy: 0.7344
Epoch 2/25
2560/2560 [==============================] - 0s 160us/step - loss: 0.9017 - accuracy: 0.7086 - val_loss: 0.6217 - val_accuracy: 0.6750
Epoch 3/25
2560/2560 [==============================] - 0s 161us/step - loss: 0.6057 - accuracy: 0.7039 - val_loss: 0.5829 - val_accuracy: 0.7359
Epoch 4/25
2560/2560 [==============================] - 0s 152us/step - loss: 0.5744 - accuracy: 0.7277 - val_loss: 0.5977 - val_accuracy: 0.7219
Epoch 5/25
2560/2560 [==============================] - 0s 146us/step - loss: 0.5879 - accuracy: 0.7223 - val_loss: 0.5855 - val_accuracy: 0.7328
Epoch 6/25
2560/2560 [==============================] - 0s 140us/step - loss: 0.5639 - accuracy: 0.7285 - val_loss: 0.5828 - val_accuracy: 0.7422
Epoch 7/25
2560/2560 [==============================] - 0s 130us/step - loss:

2021-08-15 16:00:04.351 | DEBUG    | __main__:<module>:63 - US_1_FB, NN, Image, Self-reported and Extracted Features: model training ended. AUC: 59.24, accuracy: 74.5
2021-08-15 16:00:04.352 | DEBUG    | __main__:<module>:50 - US_1_FB, LR, Image, Self-reported and Extracted Features: model training started.
2021-08-15 16:00:10.284 | DEBUG    | __main__:<module>:63 - US_1_FB, LR, Image, Self-reported and Extracted Features: model training ended. AUC: 70.39, accuracy: 74.25
 78%|███████▊  | 7/9 [08:58<02:19, 69.76s/it]2021-08-15 16:00:12.332 | DEBUG    | __main__:<module>:37 - Started the script for US_1_dating.
2021-08-15 16:00:12.335 | DEBUG    | __main__:<module>:50 - US_1_dating, NN, Image Features: model training started.


Train on 2560 samples, validate on 640 samples
Epoch 1/25
2560/2560 [==============================] - 1s 325us/step - loss: 0.7229 - accuracy: 0.5672 - val_loss: 0.6814 - val_accuracy: 0.5969
Epoch 2/25
2560/2560 [==============================] - 0s 127us/step - loss: 0.6945 - accuracy: 0.5645 - val_loss: 0.6740 - val_accuracy: 0.6062
Epoch 3/25
2560/2560 [==============================] - 0s 128us/step - loss: 0.6841 - accuracy: 0.5668 - val_loss: 0.6717 - val_accuracy: 0.6094
Epoch 4/25
2560/2560 [==============================] - 0s 123us/step - loss: 0.6789 - accuracy: 0.5734 - val_loss: 0.6704 - val_accuracy: 0.6109
Epoch 5/25
2560/2560 [==============================] - 0s 125us/step - loss: 0.6749 - accuracy: 0.5848 - val_loss: 0.6694 - val_accuracy: 0.6109
Epoch 6/25
2560/2560 [==============================] - 0s 127us/step - loss: 0.6715 - accuracy: 0.5895 - val_loss: 0.6683 - val_accuracy: 0.6109
Epoch 7/25
2560/2560 [==============================] - 0s 125us/step - loss:

...

2021-08-15 16:01:42.361 | DEBUG    | __main__:<module>:63 - US_0_FB, NN, Image and Self Reported Features: model training ended. AUC: 69.73, accuracy: 70.63
2021-08-15 16:01:42.361 | DEBUG    | __main__:<module>:50 - US_0_FB, LR, Image and Self Reported Features: model training started.
2021-08-15 16:01:48.711 | DEBUG    | __main__:<module>:63 - US_0_FB, LR, Image and Self Reported Features: model training ended. AUC: 63.98, accuracy: 66.25
2021-08-15 16:01:48.713 | DEBUG    | __main__:<module>:50 - US_0_FB, NN, Image and Extracted Features: model training started.


Train on 2560 samples, validate on 640 samples
Epoch 1/25
2560/2560 [==============================] - 1s 365us/step - loss: 0.8903 - accuracy: 0.5875 - val_loss: 0.6348 - val_accuracy: 0.6766
Epoch 2/25
2560/2560 [==============================] - 0s 140us/step - loss: 0.6088 - accuracy: 0.7070 - val_loss: 0.6249 - val_accuracy: 0.6781
Epoch 3/25
2560/2560 [==============================] - 0s 131us/step - loss: 0.5953 - accuracy: 0.7113 - val_loss: 0.6165 - val_accuracy: 0.6781
Epoch 4/25
2560/2560 [==============================] - 0s 129us/step - loss: 0.5894 - accuracy: 0.7152 - val_loss: 0.6247 - val_accuracy: 0.6875
Epoch 5/25
2560/2560 [==============================] - 0s 129us/step - loss: 0.5749 - accuracy: 0.7137 - val_loss: 0.6087 - val_accuracy: 0.6750
Epoch 6/25
2560/2560 [==============================] - 0s 126us/step - loss: 0.5648 - accuracy: 0.7199 - val_loss: 0.6114 - val_accuracy: 0.6859
Epoch 7/25
2560/2560 [==============================] - 0s 127us/step - loss:

2021-08-15 16:01:58.660 | DEBUG    | __main__:<module>:63 - US_0_FB, NN, Image and Extracted Features: model training ended. AUC: 63.0, accuracy: 70.25
2021-08-15 16:01:58.661 | DEBUG    | __main__:<module>:50 - US_0_FB, LR, Image and Extracted Features: model training started.
2021-08-15 16:02:04.501 | DEBUG    | __main__:<module>:63 - US_0_FB, LR, Image and Extracted Features: model training ended. AUC: 72.1, accuracy: 71.5
2021-08-15 16:02:04.503 | DEBUG    | __main__:<module>:50 - US_0_FB, NN, Image, Self-reported and Extracted Features: model training started.


Train on 2560 samples, validate on 640 samples
Epoch 1/25
2560/2560 [==============================] - 1s 342us/step - loss: 1.0915 - accuracy: 0.5594 - val_loss: 0.6698 - val_accuracy: 0.6875
Epoch 2/25
2560/2560 [==============================] - 0s 159us/step - loss: 0.6312 - accuracy: 0.6969 - val_loss: 0.6342 - val_accuracy: 0.6687
Epoch 3/25
2560/2560 [==============================] - 0s 162us/step - loss: 0.6038 - accuracy: 0.6988 - val_loss: 0.6338 - val_accuracy: 0.6828
Epoch 4/25
2560/2560 [==============================] - 0s 139us/step - loss: 0.5992 - accuracy: 0.7027 - val_loss: 0.6316 - val_accuracy: 0.6703
Epoch 5/25
2560/2560 [==============================] - 0s 129us/step - loss: 0.5917 - accuracy: 0.7113 - val_loss: 0.6296 - val_accuracy: 0.6719
Epoch 6/25
2560/2560 [==============================] - 0s 173us/step - loss: 0.5887 - accuracy: 0.7074 - val_loss: 0.6319 - val_accuracy: 0.6891
Epoch 7/25
2560/2560 [==============================] - 0s 155us/step - loss:

2021-08-15 16:02:15.180 | DEBUG    | __main__:<module>:63 - US_0_FB, NN, Image, Self-reported and Extracted Features: model training ended. AUC: 62.42, accuracy: 70.13
2021-08-15 16:02:15.181 | DEBUG    | __main__:<module>:50 - US_0_FB, LR, Image, Self-reported and Extracted Features: model training started.
2021-08-15 16:02:20.934 | DEBUG    | __main__:<module>:63 - US_0_FB, LR, Image, Self-reported and Extracted Features: model training ended. AUC: 72.18, accuracy: 71.88
100%|██████████| 9/9 [11:09<00:00, 74.36s/it]
2021-08-15 16:02:20.951 | DEBUG    | utils.utils:save_results:144 - Results Saved.
2021-08-15 16:02:20.952 | DEBUG    | __main__:<module>:68 - Script for Self_Reported_Segments finished.
